In [78]:
import numpy as np
import matplotlib.pyplot as plt
from random import shuffle
import time
import tkinter

In [79]:
# Kartenfarbe: 0 = Karo; 1 = Herz; 2 = Pik; 3 = Kreuz
# Kartenbezeichnung: 0 = 7; 1 = 8; 2 = 9; 3 = 10; 4 = Bube, 5 = Dame; 6 = König, 7 = Ass
card_values = [7, 8, 9, 10, 10, 10, 10, 11]
CorX=[290, 420, 550, 290, 420, 550, 30, 160, 290, 550, 680, 810]
CorY=[420, 420, 420, 220, 220, 220, 20, 20, 20, 20, 20, 20]

In [80]:
deck= []
for i in range(4):
    for j in range(8):
        deck.append((i,j))
shuffle(deck)

In [81]:
hand = deck[:3]
oponent_Hand_1 = deck[3:6]
oponent_hand_2 = deck[6:9]
open_cards = deck[9:12]
remainder = deck[12:]

In [82]:
def hand_sum(hand):
    points = np.zeros(4)
    for elem in hand:
        points[elem[0]] += card_values[elem[1]]
    return np.max(points)

In [83]:
def turn(hand, open_cards):
    best_card = [hand_sum(hand), None, None]
    for card in open_cards:
        for position in range(3):
            test_hand = hand.copy()
            test_hand[position] = card
            handsum = hand_sum(test_hand)
            if handsum > best_card[0]:
                best_card[0], best_card[1], best_card[2] = handsum, open_cards.index(card), position
    if best_card[1] != None:
        hand[best_card[2]], open_cards[best_card[1]] = open_cards[best_card[1]], hand[best_card[2]]
    if hand_sum(hand) >= 31:
        return True
    return 

In [84]:
no_action = 0
players = [hand, oponent_Hand_1, oponent_hand_2]
window = tkinter.Tk()
window.title("FIVK")
window.configure(background="green")
label = []
for i in range(12):
    label.append(i)
    img = tkinter.PhotoImage(file= "PNG-cards-1.3\\48.png")
    label[i] = tkinter.Label(image=img)
    label[i].photo = img
    label[i].place(x = CorX[i], y = CorY[i])
def update():
    for i in range(3):
        global no_action
        global open_cards
        if no_action == 3:
            print("Schiebung")
            open_cards, remainder[:3] = remainder[:3], open_cards
            shuffle(remainder)
            no_action = 0
        open_cards_before = open_cards.copy()
        print(players[i%3], i%3 + 1,  hand_sum(players[i%3]))
        print(open_cards, "Offenekarten")
        outcome = turn(players[i%3], open_cards)
        if outcome == True:
            print(f'Player {i%3 + 1} wins.')
            Hand = hand + open_cards + oponent_Hand_1 + oponent_hand_2
            cards = [] 
            for card in Hand:
                cards.append(f'PNG-cards-1.3\\{card[0]}{card[1]}.png')
            for i in range(12):
                img = tkinter.PhotoImage(file = cards[i])
                label[i].configure(image=img)
                label[i].photo = img
                return
        if open_cards == open_cards_before:
            no_action += 1
        Hand = hand + open_cards
        cards = [] 
        for card in Hand:
            cards.append(f'PNG-cards-1.3\\{card[0]}{card[1]}.png')
        for i in range(6):
            img = tkinter.PhotoImage(file = cards[i])
            label[i].configure(image=img)
            label[i].photo = img
        window.after(1000, update)


        
update()
window.mainloop()


[(2, 1), (1, 1), (0, 7)] 1 11.0
[(2, 0), (1, 2), (0, 6)] Offenekarten
[(3, 5), (1, 4), (3, 7)] 2 21.0
[(2, 0), (1, 2), (2, 1)] Offenekarten
[(1, 3), (2, 3), (3, 4)] 3 10.0
[(2, 0), (1, 2), (2, 1)] Offenekarten
[(0, 6), (1, 1), (0, 7)] 1 21.0
[(2, 0), (2, 3), (2, 1)] Offenekarten
[(3, 5), (1, 4), (3, 7)] 2 21.0
[(2, 0), (2, 3), (2, 1)] Offenekarten
Schiebung
[(1, 3), (1, 2), (3, 4)] 3 19.0
[(2, 4), (1, 5), (0, 4)] Offenekarten
[(0, 6), (1, 1), (0, 7)] 1 21.0
[(2, 4), (3, 4), (0, 4)] Offenekarten
Player 1 wins.
[(3, 5), (1, 4), (3, 7)] 2 21.0
[(2, 4), (3, 4), (1, 1)] Offenekarten
Player 2 wins.
[(1, 3), (1, 2), (1, 5)] 3 29.0
[(2, 4), (1, 4), (1, 1)] Offenekarten
[(0, 6), (0, 4), (0, 7)] 1 31.0
[(2, 4), (1, 2), (1, 1)] Offenekarten
Player 1 wins.
[(3, 5), (3, 4), (3, 7)] 2 31.0
[(2, 4), (1, 2), (1, 1)] Offenekarten
Player 2 wins.
[(1, 3), (1, 4), (1, 5)] 3 30.0
[(2, 4), (1, 2), (1, 1)] Offenekarten
Schiebung
[(0, 6), (0, 4), (0, 7)] 1 31.0
[(3, 6), (0, 1), (1, 7)] Offenekarten
Player 1 w

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.1264.0_x64__qbz5n2kfra8p0\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.1264.0_x64__qbz5n2kfra8p0\Lib\tkinter\__init__.py", line 861, in callit
    func(*args)
  File "C:\Users\alexa\AppData\Local\Temp\ipykernel_14932\2846977248.py", line 44, in update
    img = tkinter.PhotoImage(file = cards[i])
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.1264.0_x64__qbz5n2kfra8p0\Lib\tkinter\__init__.py", line 4129, in __init__
    Image.__init__(self, 'photo', name, cnf, master, **kw)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.1264.0_x64__qbz5n2kfra8p0\Lib\tkinter\__init__.py", line 4074, in __init__
    se

[(0, 6), (0, 4), (0, 7)] 1 31.0
[(1, 6), (3, 6), (2, 6)] Offenekarten
Player 1 wins.
Schiebung
[(3, 5), (3, 4), (3, 7)] 2 31.0
[(2, 5), (3, 3), (1, 2)] Offenekarten
Player 2 wins.
[(1, 7), (1, 4), (1, 5)] 3 31.0
[(2, 5), (3, 3), (1, 2)] Offenekarten
Player 3 wins.
[(0, 6), (0, 4), (0, 7)] 1 31.0
[(2, 5), (3, 3), (1, 2)] Offenekarten
Player 1 wins.
Schiebung
[(3, 5), (3, 4), (3, 7)] 2 31.0
[(0, 5), (1, 1), (1, 0)] Offenekarten
Player 2 wins.
[(1, 7), (1, 4), (1, 5)] 3 31.0
[(0, 5), (1, 1), (1, 0)] Offenekarten
Player 3 wins.
[(0, 6), (0, 4), (0, 7)] 1 31.0
[(0, 5), (1, 1), (1, 0)] Offenekarten
Player 1 wins.
Schiebung
[(3, 5), (3, 4), (3, 7)] 2 31.0
[(2, 1), (1, 3), (0, 2)] Offenekarten
Player 2 wins.
[(1, 7), (1, 4), (1, 5)] 3 31.0
[(2, 1), (1, 3), (0, 2)] Offenekarten
Player 3 wins.
[(0, 6), (0, 4), (0, 7)] 1 31.0
[(2, 1), (1, 3), (0, 2)] Offenekarten
Player 1 wins.
Schiebung
[(3, 5), (3, 4), (3, 7)] 2 31.0
[(0, 1), (0, 0), (2, 5)] Offenekarten
Player 2 wins.
[(1, 7), (1, 4), (1, 5)] 